# Homework 2

# Set up

## Installing packages

In [14]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

## Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `VERTEX_API_KEY`.


1.   Look for the key icon on the left panel of your colab.
2.   Under `Name`, create `VERTEX_API_KEY`.
3. Copy your key to `Value`.

If you cannot use VERTEX_API_KEY, you can use deepseek models via `DEEPSEEK_API_KEY`. It does not affect your score.



In [15]:
from google.colab import userdata
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')

# Download sample CVs

## Downloading sample_cv.pdf
The codes below download the sample CV


In [16]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

Retrieving folder contents


Processing file 1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp CV_1.pdf
Processing file 16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs CV_2.pdf
Processing file 15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr CV_3.pdf
Processing file 1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk CV_4.pdf
Processing file 1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C CV_5.pdf


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1NR1RUKx4GyM7QOBxKXkfh4e8jUkxFCsp
To: /content/downloaded_cvs/CV_1.pdf
100%|██████████| 147k/147k [00:00<00:00, 38.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=16lrd-uO8AAnCnv7UG9Rs_Nk6SUu0Iwbs
To: /content/downloaded_cvs/CV_2.pdf
100%|██████████| 75.1k/75.1k [00:00<00:00, 7.00MB/s]
Downloading...
From: https://drive.google.com/uc?id=15hVEuBan_EKhEty2aZBd6rcpDpP4o7Vr
To: /content/downloaded_cvs/CV_3.pdf
100%|██████████| 72.0k/72.0k [00:00<00:00, 43.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y2w_mAUEhg4vZBdvvR-0n3Jf2mKuGDRk
To: /content/downloaded_cvs/CV_4.pdf
100%|██████████| 73.3k/73.3k [00:00<00:00, 35.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLwkva-pdua6ZVvmLg9mxHeljq9D8C_C
To: /content/downloaded_cvs/CV_5.pdf
100%|██████████| 97.9k/97.9k [00:00<00:00, 44.0MB/s]
Download complete

['downloaded_cvs/CV_1.pdf',
 'downloaded_cvs/CV_2.pdf',
 'downloaded_cvs/CV_3.pdf',
 'downloaded_cvs/CV_4.pdf',
 'downloaded_cvs/CV_5.pdf']

In [17]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


📄 CV_1.pdf
|     |     |     |     | John         |           | Smith        |                   |     |     |
| --- | --- | --- | --- | ------------ | --------- | ------------ | ----------------- | --- | --- |
|     |     |     |     | Marketing    |           | Professional |                   |     |     |
|     |     |     |     | + Singapore, | Singapore |              | (cid:209) Kowloon |     |     |
Experience
|                |                  |     |          |                     |              |            |     | 2020 – | Present |
| -------------- | ---------------- | --- | -------- | ------------------- | ------------ | ---------- | --- | ------ | ------- |
| Engineer,      | ByteDance        |     |          |                     |              |            |     |        |         |
| • Worked       | in a fast-paced, |     | global   | technology          | environment. |            |     |        |         |
| • Collaborated | across           |     | teams to | sup

# Connect to our MCP server

Documentation about MCP: https://modelcontextprotocol.io/docs/getting-started/intro.

Using MCP servers in Langchain https://docs.langchain.com/oss/python/langchain/mcp.

## Check which tools that the MCP server provide

In [18]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

search_facebook_users
Search for Facebook users by display name (supports partial and fuzzy matching).

Args:
    q: Search query string (case-insensitive, matches any part of display name)
       Examples: "John", "john smith", "Smith"
    limit: Maximum number of results to return (default: 20, max: 20)
    fuzzy: Enable fuzzy matching if exact search returns no results (default: True)

Returns:
    List of user dictionaries, each containing:
    - id (int): Unique Facebook user ID for use with get_facebook_profile()
    - display_name (str): User's Facebook display name (may differ from legal name)
    - city (str): Current city of residence
    - country (str): Country of residence
    - match_type (str): "exact" or "fuzzy" (indicates search method used)
    
    Returns empty list [] if no matches found.

Example:
    search_facebook_users("Alex Chan", limit=5)
    → [{"id": 123, "display_name": "Alex Chan", "city": "Hong Kong", "country": "Hong Kong", "match_type": "exact"}]
    

## A simple agent using tools from the MCP server


In [23]:
import os
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient

# ---------------------------
# 1. Define a local tool
# ---------------------------
@tool
def say_hello(name: str) -> str:
    """Say hello to a person by name."""
    return f"Hello, {name}! 👋"

# ---------------------------
# 2. Load MCP tools + merge
# ---------------------------
client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
tools = mcp_tools + [say_hello]

# ---------------------------
# 3. Initialize Gemini (tool-enabled) or deepseek
# ---------------------------

from langchain_openai import ChatOpenAI
DEEPSEEK_API_KEY = userdata.get("DEEPSEEK_API_KEY")
llm = ChatOpenAI(
    model="deepseek-chat",
    api_key=DEEPSEEK_API_KEY,
    base_url="https://api.deepseek.com/v1",
     temperature=0,
)

llm_with_tools = llm.bind_tools(tools)

# ---------------------------
# 4. Single-step invocation
# ---------------------------
query = "Say hello to Bao using tool, then search for someone named Alice on Facebook."

response = llm_with_tools.invoke([
    HumanMessage(content=query)
])

print(response)

content="I'll help you with that! First, let me say hello to Bao, and then search for Alice on Facebook." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 2940, 'total_tokens': 3008, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 2940}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '4babdd5d-dcd2-4b90-af8d-0c44d988fce2', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c9805-f95f-7ca2-a449-9608cfe4d8c9-0' tool_calls=[{'name': 'say_hello', 'args': {'name': 'Bao'}, 'id': 'call_00_juyqGpFA6awSC6Cp6Sp1x7P6', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 2940, 'output_tokens': 68, 'total_tokens': 3008, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}


In [25]:
# This block provides you some tests to get faminilar with our MCP server

# # Test 1: Search Facebook users (exact match)
await tools[0].ainvoke({'q': "Alex Chan", 'limit': 5})

# # Test 2: Search Facebook users (fuzzy match with typo)
await tools[0].ainvoke({'q': "Alx Chn", 'limit': 5, 'fuzzy': True})

# # Test 3: Get Facebook profile
await tools[1].ainvoke({'user_id': 123})

# # Test 4: Get Facebook mutual friends
await tools[2].ainvoke({'user_id_1': 123, 'user_id_2': 456})

# # Test 5: Search LinkedIn people (exact match)
await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5})

# # Test 6: Search LinkedIn people (fuzzy match with typo)
await tools[3].ainvoke({'q': "Python", 'location': "Hong Kong", 'limit': 5, 'fuzzy': True})

# # Test 7: Get LinkedIn profile
await tools[4].ainvoke({'person_id': 456})

# Test 8: Get LinkedIn interactions
await tools[5].ainvoke({'person_id': 456})

[{'type': 'text',
  'text': '{"profile_id":456,"post_count":4,"total_likes":5,"liked_by":[4390,3622,7500,4269,8464],"engagement_score":1.25}',
  'id': 'lc_148535e4-4768-4ffa-abe9-15d6f8319833'}]

In [31]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
TOOLS = {t.name: t for t in mcp_tools}

print("✅ TOOLS ready. Loaded:", list(TOOLS.keys()))


✅ TOOLS ready. Loaded: ['search_facebook_users', 'get_facebook_profile', 'get_facebook_mutual_friends', 'search_linkedin_people', 'get_linkedin_profile', 'get_linkedin_interactions']


In [39]:
# =====================================================
#  CV Verification Main Pipeline (MULTI-RECALL UPDATED)
# =====================================================

import json, re
from rapidfuzz import fuzz

def _clean(s):
    return (s or "").strip()

def _words(s: str):
    return [w for w in re.split(r"\s+", _clean(s)) if w]

def _name_variants(name: str):
    """
    Generate multiple query variants for higher recall.
    """
    name = _clean(name)
    ws = _words(name)
    variants = []
    if name:
        variants.append(name)
    # First Last (first two tokens)
    if len(ws) >= 2:
        variants.append(f"{ws[0]} {ws[-1]}")
        variants.append(f"{ws[0]} {ws[1]}")
    # Last First
    if len(ws) >= 2:
        variants.append(f"{ws[-1]} {ws[0]}")
    # Just first+last without punctuation
    variants = [re.sub(r"[^A-Za-z\s\-']", " ", v).strip() for v in variants]
    variants = [re.sub(r"\s+", " ", v) for v in variants if v]
    # de-dup
    seen = set()
    out = []
    for v in variants:
        if v.lower() not in seen:
            seen.add(v.lower())
            out.append(v)
    return out

# 1️⃣ 用 LLM 抽取 CV 信息（要求严格 JSON）
async def extract_candidate_from_cv(cv_text: str) -> dict:
    prompt = f"""
You are a strict information extractor for resume/CV.
Return STRICT JSON only (no markdown, no explanation).

Fields:
- name: candidate full name (NOT email, NOT 'Resume', NOT 'CV')
- city: string|null
- country: string|null
- education: list of {{school, degree, field, start_year, end_year}}
- experience: list of {{company, title, start_year, end_year}}

If missing, use null or [].

CV TEXT:
{cv_text[:12000]}
"""
    msg = await llm.ainvoke(prompt)
    text = msg.content if hasattr(msg, "content") else str(msg)

    m = re.search(r"\{.*\}", text, flags=re.S)
    if not m:
        return {"name": None, "city": None, "country": None, "education": [], "experience": []}

    try:
        data = json.loads(m.group(0))
    except:
        return {"name": None, "city": None, "country": None, "education": [], "experience": []}

    data.setdefault("name", None)
    data.setdefault("city", None)
    data.setdefault("country", None)
    data.setdefault("education", [])
    data.setdefault("experience", [])

    # name 清洗
    if isinstance(data.get("name"), str):
        n = data["name"].strip()
        if ("@" in n) or (n.lower() in {"resume", "cv", "curriculum vitae"}):
            data["name"] = None

    return data

def normalize_li_results(li_results):
    if li_results is None:
        return []
    if isinstance(li_results, list):
        out = []
        for r in li_results:
            if isinstance(r, dict) and "id" in r:
                rr = dict(r)
                try:
                    rr["id"] = int(rr["id"])
                except Exception:
                    continue
                out.append(rr)
        return out
    if isinstance(li_results, dict) and "content" in li_results:
        return normalize_li_results(li_results["content"])
    return []

def _set_lower(xs):
    return {(_clean(x)).lower() for x in xs if _clean(x)}

def rank_linkedin_results(results, cand):
    """
    Stronger rerank using:
    - name similarity
    - location overlap
    - company overlap (CV vs result headline)
    - school overlap (CV schools vs result headline)
    """
    name = _clean(cand.get("name"))
    city = _clean(cand.get("city"))
    country = _clean(cand.get("country"))

    cv_companies = _set_lower([e.get("company") for e in cand.get("experience", []) if isinstance(e, dict)])
    cv_schools = _set_lower([e.get("school") for e in cand.get("education", []) if isinstance(e, dict)])

    scored = []
    for r in results:
        s = 0.0
        r_name = r.get("name", "")
        r_loc = (r.get("location") or "").lower()
        r_head = (r.get("headline") or "").lower()

        s += fuzz.token_set_ratio(name, r_name) * 0.65

        if city and city.lower() in r_loc:
            s += 10
        if country and country.lower() in r_loc:
            s += 6

        # company/school weak match via headline text (cheap but works)
        for c in list(cv_companies)[:3]:
            if c and c in r_head:
                s += 6
        for sch in list(cv_schools)[:2]:
            if sch and sch in r_head:
                s += 4

        scored.append((s, r))

    scored.sort(key=lambda x: x[0], reverse=True)
    return scored[0][1]

def compare_cv_vs_linkedin(cand, li_prof):
    issues = []

    if cand.get("country") and li_prof.get("country"):
        if cand["country"].lower() != li_prof["country"].lower():
            issues.append("Country mismatch")

    cv_comp = _set_lower([e.get("company") for e in cand.get("experience", []) if isinstance(e, dict)])
    li_comp = _set_lower([e.get("company") for e in li_prof.get("experience", []) if isinstance(e, dict)])
    if cv_comp and li_comp and not cv_comp.intersection(li_comp):
        issues.append("Company mismatch")

    cv_sch = _set_lower([e.get("school") for e in cand.get("education", []) if isinstance(e, dict)])
    li_sch = _set_lower([e.get("school") for e in li_prof.get("education", []) if isinstance(e, dict)])
    if cv_sch and li_sch and not cv_sch.intersection(li_sch):
        issues.append("Education mismatch")

    return issues

def issues_to_score(issues, found_profile: bool):
    # 没找到 profile：给一个“中等偏高”的风险，但不要全部一样拉满
    if not found_profile:
        return 0.55
    if not issues:
        return 0.1
    if len(issues) == 1:
        return 0.6
    return 0.9

async def _linkedin_search_multi(query_list, cand, limit=20):
    """
    Try multiple queries and merge results by id.
    """
    merged = {}
    for q in query_list:
        if not _clean(q):
            continue
        res = await TOOLS["search_linkedin_people"].ainvoke({
            "q": q,
            "location": None,
            "industry": None,
            "limit": limit,
            "fuzzy": True
        })
        res = normalize_li_results(res)
        for r in res:
            merged[r["id"]] = r
    return list(merged.values())

# 5️⃣ 主 verify 函数（多策略召回）
async def verify_one_cv(cv_text: str):

    cand = await extract_candidate_from_cv(cv_text)
    name = _clean(cand.get("name"))

    if not name:
        return {"issues": ["Cannot extract name"], "score": 0.9}

    # DEBUG：先看看抽取出来的 name 是否正常（跑通后可删）
    # print("EXTRACTED:", cand.get("name"), cand.get("city"), cand.get("country"))

    # (A) 先用 name 的多种变体去搜
    queries = _name_variants(name)
    li_results = await _linkedin_search_multi(queries, cand, limit=20)

    # (B) 如果还是空：用公司/职位关键词做召回（非常重要）
    if not li_results:
        exp = [e for e in cand.get("experience", []) if isinstance(e, dict)]
        # 取前2个公司+职位拼关键词
        fallback_queries = []
        for e in exp[:2]:
            comp = _clean(e.get("company"))
            title = _clean(e.get("title"))
            if comp and title:
                fallback_queries.append(f"{comp} {title}")
            elif comp:
                fallback_queries.append(comp)
            elif title:
                fallback_queries.append(title)

        # 再加一个“纯职位关键词”兜底
        if exp and _clean(exp[0].get("title")):
            fallback_queries.append(_clean(exp[0].get("title")))

        # 去重
        fq = []
        seen = set()
        for x in fallback_queries:
            if x and x.lower() not in seen:
                seen.add(x.lower())
                fq.append(x)

        if fq:
            li_results = await _linkedin_search_multi(fq, cand, limit=20)

    # DEBUG：打印每份 CV 最终召回数量（跑通后可删）
    # print("DEBUG queries:", queries, "->", len(li_results))

    if not li_results:
        return {"issues": ["No LinkedIn profile found"], "score": issues_to_score([], found_profile=False)}

    best = rank_linkedin_results(li_results, cand)

    # ✅ person_id 必须 int
    person_id = best.get("id")
    try:
        person_id = int(person_id)
    except Exception:
        return {"issues": [f"Invalid LinkedIn id: {person_id}"], "score": 0.9}

    li_prof = await TOOLS["get_linkedin_profile"].ainvoke({"person_id": person_id})

    issues = compare_cv_vs_linkedin(cand, li_prof)
    score = issues_to_score(issues, found_profile=True)

    return {
        "candidate": cand,
        "linkedin": li_prof,
        "issues": issues,
        "score": score
    }


In [40]:
results = []
scores = []

for item in all_cvs:
    r = await verify_one_cv(item["text"])
    results.append(r)
    scores.append(r["score"])
    print(item["file"], r["score"], r["issues"])

print("Scores:", scores)


CV_1.pdf 0.55 ['No LinkedIn profile found']
CV_2.pdf 0.55 ['No LinkedIn profile found']
CV_3.pdf 0.55 ['No LinkedIn profile found']
CV_4.pdf 0.55 ['No LinkedIn profile found']
CV_5.pdf 0.55 ['No LinkedIn profile found']
Scores: [0.55, 0.55, 0.55, 0.55, 0.55]


# Evaluation code

In the test phase, you will be given 5 CV files with fixed names:

    CV_1.pdf, CV_2.pdf, CV_3.pdf, CV_4.pdf, CV_5.pdf

Your system must process these CVs and output a list of 5 scores,
one score per CV, in the same order:

    scores = [s1, s2, s3, s4, s5]

Each score must be a float in the range [0, 1], representing the
reliability or confidence that the CV is valid (or meets the task criteria).

The ground-truth labels are binary:

    groundtruth = [0 or 1, ..., 0 or 1]

Each CV is evaluated independently using a threshold of 0.5:

- If score > 0.5 and groundtruth == 1 → Full credit
- If score ≤ 0.5 and groundtruth == 0 → Full credit
- Otherwise → No credit

In other words, 0.5 is the decision threshold.

- Each CV contributes equally.
- Final score = (number of correct decisions) / 5


In [41]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [27]:
scores = ... # Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


TypeError: object of type 'ellipsis' has no len()

In [42]:
results = []
scores = []

for item in all_cvs:
    r = await verify_one_cv(item["text"])
    results.append(r)
    scores.append(r["score"])

print("Generated scores:", scores)


Generated scores: [0.55, 0.55, 0.55, 0.55, 0.55]
